In [ ]:
from hagis import Mapper, Point

url = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/USA/MapServer/0"

class City:
    objectid: int
    areaname: str
    pop2000: int
    shape: Point

mapper = Mapper(url, City)

for city in mapper.query("pop2000 > 1000000", orderByFields="pop2000 DESC"):
    print(city.areaname, city.pop2000, city.shape.x, city.shape.y)


In [ ]:
from hagis import Mapper

url = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/USA/MapServer/0"

# Not specifying the target type results in "SimpleNamespace" objects.
mapper = Mapper(url)

# Keyword arguments such as "outSR" are also sent to the Esri API.
for city in mapper.query("pop2000 > 1000000", orderByFields="pop2000 DESC", outSR=102100):
    print(city.areaname, city.pop2000, city.shape.x, city.shape.y)

# Contains all fields.
city.__dict__


In [ ]:
from hagis import Mapper, Polyline
from dataclasses import dataclass

url = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/USA/MapServer/1"

# Optionally supports dataclass.
@dataclass(frozen=True)
class Highway:
    objectid: int
    type: str
    name: str # This is actually called "route" in the table.
    shape: Polyline

mapper = Mapper(url, Highway, mapping={"name": "route"})

for highway in mapper.query("route = 'Interstate  55'"):
    print(highway.objectid, highway.name, highway.type)


In [ ]:
from hagis import Mapper
from datetime import datetime
from itertools import islice

url = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/SpatioTemporalAggregation/RainfallTimeSeriesDataIllinois/MapServer/7"

class Rain:
    objectid: int
    site_no: str
    date_time: datetime
    rainfall_inch: float

# Non-spatial table.
mapper = Mapper(url, Rain)

# Take only 10 records (otherwise keeps yielding).
for rain in islice(mapper.query(), 10):
    print(rain.__dict__)


In [ ]:
from hagis import Mapper

url = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/RedlandsEmergencyVehicles/FeatureServer/0"

mapper = Mapper(url)

mapper.delete("unitname LIKE '%Hagis%'")


In [ ]:
from hagis import Mapper

url = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/RedlandsEmergencyVehicles/FeatureServer/0"

[item for item in Mapper(url).query("unitname LIKE '%Hagis%'")]


In [ ]:
from hagis import Mapper, Point

url = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/RedlandsEmergencyVehicles/FeatureServer/0"

class Ambulance:
    objectid: int
    unitname: str
    type: int
    speed: float
    shape: Point

mapper = Mapper(url, Ambulance)

ambulance = Ambulance()
ambulance.unitname = "Created by https://github.com/jshirota/Hagis"
ambulance.type = 1
ambulance.speed = 12.34567
ambulance.shape = Point()
ambulance.shape.x, ambulance.shape.y = -120, 50

# Returns the newly created object ids.
oid = mapper.insert([ambulance])[0]

mapper.find(oid).__dict__
